# Bootstrapping model fits
The previous section describes fitting a single model.
But we may also want to have confidence estimates for the fit.
We can do that via bootstrapping the data set.

The overall recommended workflow is to first fit models to all the data to determine the number of epitopes, etc.
Then once the desired fitting parameters are determined, you can bootstrap to get confidence on predictions.

Here we illustrate bootstrapping on the simulated RBD data using the noisy data with an average of 2 mutations per gene.

## Get a model fit to all the data
The first step is just to fit a `Polyclonal` model to all the data.
We just did that in the previous notebook for our RBD example and saved the model using [pickle](https://docs.python.org/3/library/pickle.html), so here we just read in that model rather than re-fitting.
We will call this the "root" model as it's used as the starting point (root) for the subsequent bootstrapping.
Note that data (which we will bootstrap) are attached to this pre-fit model:

In [1]:
# NBVAL_IGNORE_OUTPUT

import pickle

with open("fit_RBD_model.pickle", "rb") as f:
    root_poly = pickle.load(f)

root_poly.data_to_fit.head()

,library,aa_substitutions,concentration,prob_escape,IC90
0,avg2muts,,0.25,0.05044,0.1128
1,avg2muts,,0.25,0.14310,0.1128
2,avg2muts,,0.25,0.05452,0.1128
3,avg2muts,,0.25,0.08473,0.1128
4,avg2muts,,0.25,0.04174,0.1128


## Create and fit bootstrapped models
To create the bootstrapped models, we initialize a `PolyclonalCollection`, here just using 5 samples for speed (for good error estimates you may want more on the order of 20 to 100).
Note it is important that the root model you are using has already been fit to the data!

In [2]:
import polyclonal

n_bootstrap_samples = 5

bootstrap_poly = polyclonal.PolyclonalCollection(
    root_polyclonal=root_poly,
    n_bootstrap_samples=n_bootstrap_samples,
)

Now fit the models:

In [3]:
# NBVAL_IGNORE_OUTPUT

import time

start = time.time()
print(f"Starting fitting bootstrap models at {time.asctime()}")
n_fit, n_failed = bootstrap_poly.fit_models()
print(f"Fitting took {time.time() - start:.3g} seconds, finished at {time.asctime()}")
assert n_failed == 0 and n_fit == n_bootstrap_samples

Starting fitting bootstrap models at Mon Mar 14 19:55:41 2022
Fitting took 101 seconds, finished at Mon Mar 14 19:57:21 2022


## Look at summarized results

In [4]:
bootstrap_poly.activity_wt_df_replicates.round(1).head()

,epitope,activity,bootstrap_replicate
0,1,1.3,1
1,2,3.2,1
2,3,1.9,1
3,1,1.2,2
4,2,3.2,2


In [5]:
bootstrap_poly.activity_wt_df.round(1)

,epitope,mean_activity,median_activity,std_activity
0,1,1.3,1.3,0.1
1,2,3.2,3.2,0.0
2,3,2.0,2.0,0.0


In [6]:
bootstrap_poly.mut_escape_df_replicates.round(1).head()

,epitope,site,wildtype,mutant,mutation,escape,bootstrap_replicate
0,1,331,N,A,N331A,0.1,1
1,1,331,N,D,N331D,0.1,1
2,1,331,N,E,N331E,0.3,1
3,1,331,N,F,N331F,0.1,1
4,1,331,N,G,N331G,2.2,1


In [7]:
bootstrap_poly.mut_escape_df.round(1).head()

,epitope,site,wildtype,mutant,mutation,mean_escape,median_escape,std_escape,n_bootstrap_replicates,frac_bootstrap_replicates
0,1,331,N,A,N331A,0.1,0.0,0.1,5,1.0
1,1,331,N,D,N331D,0.5,0.1,0.5,5,1.0
2,1,331,N,E,N331E,0.0,-0.1,0.4,5,1.0
3,1,331,N,F,N331F,0.3,0.2,0.3,5,1.0
4,1,331,N,G,N331G,2.0,2.0,0.5,5,1.0


In [8]:
bootstrap_poly.mut_escape_site_summary_df_replicates.round(1).head()

,epitope,site,wildtype,mean,total positive,max,min,total negative,bootstrap_replicate
0,1,331,N,0.7,11.7,2.2,-0.0,-0.0,1
1,1,332,I,0.8,15.6,2.5,-0.6,-1.1,1
2,1,333,T,0.4,7.3,1.4,-0.5,-0.9,1
3,1,334,N,0.7,12.6,2.3,-0.2,-0.2,1
4,1,335,L,0.1,5.4,1.4,-1.4,-3.7,1
